In [1]:
import pandas as pd 
df=pd.read_csv('TV Shows.csv')

In [2]:
import pyodbc

def split_data(df, p1_percent=0.4, p2_percent=0.3, conn_str=None, db_table=None):
    total_rows = df.shape[0]

    part1 = int(total_rows * p1_percent)
    part2 = int(total_rows * (p1_percent + p2_percent))

    json_file = df.iloc[:part1]
    csv_file = df.iloc[part1:part2]
    db_table = df.iloc[part2:]

    json_file.to_json('output.json', orient='records', lines=True)
    csv_file.to_csv('output.csv', index=False, header=False)

    if conn_str and db_table:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()

        insert_query = f"INSERT INTO {db_table} (Title, Year, Duration, Category, Score, Cast, Synopsis) VALUES ({', '.join(['?'] * 7)})"
        for _, row in db_table.iterrows():
            cursor.execute(insert_query, tuple(row[['Title', 'Year', 'Duration', 'Category', 'Score', 'Cast', 'Synopsis']]))

        conn.commit()
        cursor.close()
        conn.close()
split_data(df, p1_percent=0.4, p2_percent=0.25, conn_str="Driver={SQL Server};Server=LAPTOP-5VUQJT8J\\SQLEXPRESS;Database=CHALLENGE;UID=;PWD=;", db_table="Synopsis")
